In [ ]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install colabgymrender==1.0.2

In [ ]:
! wget http://www.atarimania.com/roms/Roms.rar
! mkdir /content/ROM/
! unrar e /content/Roms.rar /content/ROM/
! python -m atari_py.import_roms /content/ROM/

In [ ]:
!pip3 install Box2D
!pip3 install box2d-py
!pip3 install gym[Box_2D]
import gym
from gym import envs
from colabgymrender.recorder import Recorder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras.optimizers import Adam

In [ ]:
env = gym.make("Acrobot-v1")

def create_model():
  inputs = layers.Input(shape=(6))
  layer1 = layers.Dense(256,activation='relu')(inputs)
  layer2 = layers.Dense(256,activation='relu')(layer1)
  output = layers.Dense(3)(layer2)

  return keras.Model(inputs=inputs,outputs=output)

model = create_model()
model.compile(optimizer=Adam(learning_rate=0.001),loss='mean_squared_error')
target_model = create_model()
target_model.compile(optimizer=Adam(learning_rate=0.001),loss='mean_squared_error')

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

model.load_weights("/content/gdrive/My Drive/Reinforcement-Learning-project/acrobot.h5")

In [ ]:

total_rewards = 0
frame_count = 0
episode_count = 0
max_memory_length = 100000
update_after = 4
update_target_after = 100
epsilon = 1
gamma = 0.99

action_history = []
state_history = []
state_next_history = []
rewards_history = []
done_history = []
episode_reward_history = []

for i in range(1002):
  state = env.reset()
  state = np.array(state)
  rewards=0

  for step in range(1,501):
    frame_count+=1

    if frame_count < 50000 or epsilon > np.random.rand(1)[0]:
      action = env.action_space.sample()
    else:
      state_tensor = tf.convert_to_tensor(state)
      state_tensor = tf.expand_dims(state_tensor,0)
      action_prob = model.predict(state_tensor)
      action = tf.argmax(action_prob[0]).numpy()

    epsilon -= 0.99/100000
    epsilon = max(epsilon,0.1)

    state_next,reward,done,info = env.step(action)
    state_next = np.array(state_next)

    rewards += reward

    action_history.append(action)
    state_history.append(state)
    state_next_history.append(state_next)
    done_history.append(done)
    rewards_history.append(reward)

    state = state_next

    if len(done_history) > 32:
      indices = np.random.choice(range(len(done_history)),size=32)

      state_sample = np.array([state_history[i] for i in indices])
      state_next_sample = np.array([state_next_history[i] for i in indices])
      rewards_sample = [rewards_history[i] for i in indices]
      action_sample = [action_history[i] for i in indices]
      done_sample = tf.convert_to_tensor([float(done_history[i]) for i in indices])

      future_rewards = target_model.predict(state_next_sample)
      current_rewards = model.predict(state_sample)
      updated_q = current_rewards[:]

      for idx,terminal in enumerate(done_sample):
        if terminal:
          future_rewards[idx]=0.0
        updated_q[idx,action_sample[idx]] = rewards_sample[idx] + gamma*tf.reduce_max(future_rewards,axis=1)[idx]

      model.train_on_batch(state_sample,updated_q)

    if frame_count % update_target_after ==0:
      target_model.set_weights(model.get_weights())

    if len(rewards_history)>max_memory_length:
      del rewards_history[:1]
      del state_history[:1]
      del state_next_history[:1]
      del action_history[:1]
      del done_history[:1]

    if done:
      break

  print("episode reward:",rewards,"mean reward",total_rewards,"episode:",episode_count,"frame:",frame_count)
  episode_reward_history.append(rewards)
  if len(episode_reward_history)>100:
    del episode_reward_history[:1]
  episode_count+=1
  total_rewards = np.mean(episode_reward_history)

  if i%200==0:
    model.save("acrobot.h5")

  if total_rewards > -100:
    print("solved at episode: ",episode_count)
    break
#env.play()

episode reward: -500.0 mean reward 0 episode: 0 frame: 500
episode reward: -500.0 mean reward -500.0 episode: 1 frame: 1000
episode reward: -500.0 mean reward -500.0 episode: 2 frame: 1500
episode reward: -500.0 mean reward -500.0 episode: 3 frame: 2000
episode reward: -500.0 mean reward -500.0 episode: 4 frame: 2500
episode reward: -500.0 mean reward -500.0 episode: 5 frame: 3000
episode reward: -500.0 mean reward -500.0 episode: 6 frame: 3500
episode reward: -500.0 mean reward -500.0 episode: 7 frame: 4000
episode reward: -500.0 mean reward -500.0 episode: 8 frame: 4500
episode reward: -500.0 mean reward -500.0 episode: 9 frame: 5000
episode reward: -500.0 mean reward -500.0 episode: 10 frame: 5500
episode reward: -500.0 mean reward -500.0 episode: 11 frame: 6000
episode reward: -500.0 mean reward -500.0 episode: 12 frame: 6500
episode reward: -500.0 mean reward -500.0 episode: 13 frame: 7000
episode reward: -500.0 mean reward -500.0 episode: 14 frame: 7500
episode reward: -500.0 mea

In [ ]:
# directory = './video'
# env = Recorder(env, directory)

for i in range(10):
  r=0
  obs = env.reset()
  for j in range(501):
    obs = np.array(obs)
    obs = tf.convert_to_tensor([obs])
    action = model.predict(obs)
    a = tf.argmax(action[0]).numpy()
    obs_,reward,done,_ = env.step(a)
    r+=reward
    if done:
      break
    obs = obs_
  print(r)

-97.0
-90.0
-88.0
-74.0
-77.0
-80.0
-78.0
-84.0
-69.0
-77.0


In [ ]:
directory = './video'
env = Recorder(env, directory)

done=False
obs = env.reset()
re=0
for i in range(500):
  obs = np.array(obs)
  obs = tf.convert_to_tensor([obs])
  action = model.predict(obs)
  a = tf.argmax(action[0]).numpy()
  obs,r,done,_ = env.step(a)
  re+=r
  if done:break
env.play()